# Computation, Problem Set #1a
## Numerical Integration

In [128]:
import math
import numpy as np
import scipy as sy

### Excercise 2.1

In [44]:
def numerical_integration(f,a,b,N,method):
    
    if method not in ['midpoint','trapezoid','simpsons']:
        raise ValueError("Mode must be 'midpoint', 'trapezoid', or 'simpsons'")
        
    if method == 'midpoint':
        h = float(b-a)/N
        value = 0.0
        for i in range(N):
            value += f((a + 0.5*h) + i*h)
        value *= h
        
    if method == 'trapezoid':
        h = float(b-a)/N
        value = 0.5*(f(a) + f(b))
        for i in range(1,N):
            value += f(a + i*h)
        value *= h
        
    if method == 'simpsons':
        h = float(b-a)/N
        value = f(a) + f(b)
        for i in range(1,2*N):
            x = a + i*h/2
            if (i % 2 == 0):
                value += 2*f(x)
            else:
                value += 4*f(x)
        value *= h/6
                
    return value

In [45]:
gx = lambda x : 0.1*x**4 - 1.5*x**3 + 0.53*x**2 + 2*x + 1

In [121]:
true_value = 4373+1/3
value_midpoint  = numerical_integration(gx,-10,10,1000,'midpoint')
value_trapezoid = numerical_integration(gx,-10,10,1000,'trapezoid')
value_simpson   = numerical_integration(gx,-10,10,1000,'simpsons')
print("(midpoint)  result : ",value_midpoint, " error :",abs(true_value-value_midpoint))
print("(trapezoid) result : ",value_trapezoid, " error :",abs(true_value-value_trapezoid))
print("(simpson)   result : ",value_simpson, " error :",abs(true_value-value_simpson))

(midpoint)  result :  4373.319646676004  error : 0.013686657329344598
(trapezoid) result :  4373.36070665599  error : 0.027373322656785604
(simpson)   result :  4373.333333336001  error : 2.6675479602999985e-09


### Exercise 2.2

In [81]:
from scipy.stats import norm

In [27]:
def newton_cotes(mu,sig,N,k):
    
    Z = np.linspace(mu-k*sig,mu+k*sig,N)
    w = np.zeros_like(Z)
    w[0] = norm.cdf((Z[0]+Z[1])/2,loc=mu,scale=sig)
    w[N-1] = 1 - norm.cdf((Z[N-2]+Z[N-1])/2,loc=mu,scale=sig)
    
    for i in range(1, N-1):
        zmin = (Z[i-1]+Z[i])/2
        zmax = (Z[i]+Z[i+1])/2
        w[i] = norm.cdf(zmax,loc=mu,scale=sig) - norm.cdf(zmin,loc=mu,scale=sig)
        
    return Z, w

In [28]:
Z, w = newton_cotes(0,1,11,3)
print("Z =",Z)
print("w =",w)

Z = [-3.  -2.4 -1.8 -1.2 -0.6  0.   0.6  1.2  1.8  2.4  3. ]
w = [0.00346697 0.01439745 0.04894278 0.11725292 0.19802845 0.23582284
 0.19802845 0.11725292 0.04894278 0.01439745 0.00346697]


### Excercise 2.3

In [29]:
def lognorm_dist(mu,sig,N,k):
    logA, w = newton_cotes(mu,sig,N,k)
    A = np.exp(logA)
    return A, w

In [30]:
A, w = lognorm_dist(0,1,11,3)
print("A =",A)
print("w =",w)

A = [ 0.04978707  0.09071795  0.16529889  0.30119421  0.54881164  1.
  1.8221188   3.32011692  6.04964746 11.02317638 20.08553692]
w = [0.00346697 0.01439745 0.04894278 0.11725292 0.19802845 0.23582284
 0.19802845 0.11725292 0.04894278 0.01439745 0.00346697]


### Excercise 2.4

In [31]:
mu = 10.5
sig = 0.8

Y, w = lognorm_dist(mu,sig,11,3)
print("Approximated expected income = ",Y@w)
print("Exact expected income = ",np.exp(mu+0.5*sig**2))

Approximated expected income =  50352.456192765916
Exact expected income =  50011.087008521754


### Excercise 3.1

In [112]:
from scipy.optimize import root

In [125]:
def gaussian_quad(f,a,b,N):

    def system_of_eq(params):
        w1, w2, w3, x1, x2, x3 = params
        w = np.array([w1,w2,w3])
        x = np.array([x1,x2,x3])
        eq = np.empty(len(w)*2)
        for i in range(1,len(w)*2+1):
            eq[i-1] = (b**i-a**i)/i - w@(x**(i-1))
        return eq
    
    params = root(system_of_eq, np.ones(6))
    w = params.x[:3]
    x = params.x[3:]
    value = np.sum(w*f(x))
    
    return value

In [126]:
gx = lambda x : 0.1*x**4 - 1.5*x**3 + 0.53*x**2 + 2*x + 1
value_gaussian_quad = gaussian_quad(gx,-10,10,1000)
print("(gaussian_quadature) result : ",value_gaussian_quad, " error :",abs(true_value-value_gaussian_quad))

(gaussian_quadature) result :  4373.333333340381  error : 7.047674444038421e-09


### Exercise 3.2

In [130]:
result = sy.integrate.quad(gx,-10,10)[0]
error = abs(true_value-result)
print("result :",result," error :",error)

result : 4373.333333333334  error : 9.094947017729282e-13


### Excercise 4.1

In [133]:
import scipy.stats as sts

def monte_carlo(f,Omega,N):
    x = np.random.uniform(Omega[0][0], Omega[0][1], N)
    y = np.random.uniform(Omega[1][0], Omega[1][1], N)
    V = (abs(Omega[0][0]-Omega[0][1]) * abs(Omega[1][0]-Omega[1][1]))
    return V * np.sum(f(x,y)) / N

In [157]:
f = lambda x, y : (x**2 + y**2) <= 1
diff = 10
N = 25
while diff > 0.0:
    N += 1
    pi = monte_carlo(f,[[-1,1],[-1,1]],N)
    diff = abs(3.1415-math.floor(pi*1e+4)/1e+4)
print("The number of random draws for the 4-th decimal accuracy: ",N, ", Pi = ",pi)

The number of random draws for the 4-th decimal accuracy:  862 , Pi =  3.1415313225058004


### End